In [1]:
# Parameterized Decorators

In [18]:
def timed(fun):
    from time import perf_counter
    def inner (*args, **kwargs):
        start = perf_counter()
        result = fun(*args, **kwargs)
        elapsed = perf_counter() - start
        print("Run time is {:.4f}ms".format(elapsed * 1000))
        return result
    return inner

In [19]:
def calc_list_product(xlist):
    from functools import reduce
    return reduce(lambda x, y: x * y, xlist)

In [20]:
calc_list_product([3,4,5])

60

In [21]:
li = [3, 2, 4]
calc_list_product = timed(calc_list_product)
calc_list_product(li)

Run time is 0.0117ms


24

In [19]:
def timed(fun):
    from time import perf_counter
    def inner (*args, **kwargs):
        elapsed = 0
        for i in range(5): # repetition = 5 is hardcoded
            start = perf_counter()
            result = fun(*args, **kwargs)
            elapsed = ((perf_counter() - start) + elapsed * i  ) / (i + 1)
        print("Run time is {:.4f}ms".format(elapsed * 1000))
        return result
    return inner

In [20]:
@timed
def calc_list_product(xlist):
    from functools import reduce
    return reduce(lambda x, y: x * y, xlist)

In [21]:
calc_list_product(li)

Run time is 0.0037ms


24

In [ ]:
# making repetitions a variable

In [22]:
def timed(fun, rep):
    from time import perf_counter
    def inner (*args, **kwargs):
        total_time = 0
        for i in range(rep): # repetition = 5 is hardcoded
            start = perf_counter()
            result = fun(*args, **kwargs)
            total_time += (perf_counter() - start)
        elapsed = total_time / rep
        print("Run time is {0:.4f}ms, {1} repetitions".format(elapsed * 1000, rep))
        return result
    return inner

In [23]:
calc_list_product = timed(calc_list_product, 3) # you cant use @ syntax

In [24]:
calc_list_product(li)

Run time is 0.0038ms
Run time is 0.0055ms
Run time is 0.0029ms
Run time is 0.0874ms, 3 repetitions


24

In [28]:
# decorator factory
# This is a function that returns a decorator

In [29]:
def dec(fn):
    print("running dec")
    
    def inner(*args):
        print("running inner")
        return(fn(*args))
    return inner

In [40]:
@dec # calls dec
def fun():
    print("Running fun()")

running dec


In [42]:
fun = dec(fun)
# if you call this code again, it will print running dec twice when you call fun()

running dec


In [43]:
fun()

running inner
running inner
Running fun()


In [50]:
def dec_factory(): # creates and returns the decorator that we want
    def dec(fn):
        print("running dec")

        def inner(*args):
            print("running inner")
            return(fn(*args))
        return inner
    return dec

In [49]:
dec = dec_factory()

In [51]:
def my_fun():
    print("HI, i am your function")

In [52]:
my_fun = dec(my_fun)

running dec


In [53]:
@dec # no function call, since dec is the decorator
def my_fun():
    print("HI, i am your function")

running dec


In [54]:
@dec_factory() # function call present, (), since dec_factory itself is not the decorator but returns the decorator
def my_fun():
    print("HI, i am your function")

running dec


In [55]:
my_fun()

running inner
HI, i am your function


In [56]:
my_fun = dec_factory()(my_fun)

running dec


In [57]:
my_fun()

running inner
running inner
HI, i am your function


In [58]:
def dec_factory(par_1, par_2): # you put parameters here
    def dec(fn):
        print("running dec")
        def inner(*args):
            print("Parameters, we have, {0} and {1}".format(par_1, par_2))
            return(fn(*args))
        return inner
    return dec

In [59]:
dec = dec_factory(10, 20)

In [60]:
@dec # arguments already put
def my_fun():
    print("HI, i am your function")

running dec


In [61]:
my_fun()

Parameters, we have, 10 and 20
HI, i am your function


In [63]:
@dec_factory(100, 200) # parameters passed from here
def my_fun():
    print("HI, i am your function")

running dec


In [64]:
my_fun()

Parameters, we have, 100 and 200
HI, i am your function


In [65]:
my_fun = dec_factory('a', 'b')(my_fun)

running dec


In [66]:
my_fun()

Parameters, we have, a and b
Parameters, we have, 100 and 200
HI, i am your function


In [68]:
# to create parameterized decorators, we do it not by modifying parameters
# to the decorator, but by creating a decorator function, an outer function
# that encompases and returns our decorator, but because of closures we can pass 
# the outer parameters to the decorator itself

In [85]:
def dec_factory(rep):
    def timed(fun):
        from time import perf_counter
        from functools import wraps
        @wraps(fun) # wraps here is also a decorator factory
        def inner (*args, **kwargs):
            elapsed = 0
            for i in range(rep): # repetition = 5 is hardcoded
                start = perf_counter()
                result = fun(*args, **kwargs)
                elapsed = ((perf_counter() - start) + elapsed * i  ) / (i + 1)
            print("Run time is {0:.4f}ms, {1} ran {2} times".format(elapsed * 1000, fun.__name__, rep))
            return result
        return inner
    return timed


In [76]:
@dec_factory(8)
def prin():
    pass

In [77]:
prin()

Run time is 0.0003ms, prin ran 8 times


In [87]:
# rename dec_factory to timed
def timed(rep):
    def dec(fun):
        from time import perf_counter
        from functools import wraps
        @wraps(fun) # wraps here is also a decorator factory
        def inner (*args, **kwargs):
            elapsed = 0
            for i in range(rep): # repetition = 5 is hardcoded
                start = perf_counter()
                result = fun(*args, **kwargs)
                elapsed = ((perf_counter() - start) + elapsed * i  ) / (i + 1)
            print("Run time is {0:.4f}ms, {1} ran {2} times".format(elapsed * 1000, fun.__name__, rep))
            return result
        return inner
    return dec



In [88]:
type(dec_factory)

function

In [89]:
type(dec)

function

In [90]:
type(timed)

function

In [91]:
timed.__closure__

In [92]:
dec.__closure__ # dec is a decorator

(<cell at 0x7f5dac5f34c8: int object at 0x8a8a00>,
 <cell at 0x7f5dac5f3f48: int object at 0x8a8b40>)

In [93]:
dec_factory.__closure__

In [94]:
@timed(2)
def calc_list_product(xlist):
    from functools import reduce
    return reduce(lambda x, y: x * y, xlist)

In [95]:
calc_list_product([2, 3, 1, 2, 4])

Run time is 0.0072ms, calc_list_product ran 2 times


48